In [1]:
from ctapipe.core import Container, Field, Map
import numpy as np

In [2]:
class SubContainer(Container):
    junk = Field("nothing","Some junk")

class EventContainer(Container):
    event_id = Field(-1,"event id number")
    tels_with_data = Field([], "list of telescopes with data")
    sub = Field(SubContainer(), "stuff")  # a sub-container in the hierarchy

    # for dicts of sub-containers, use Map instead 
    # of a dict() as the default value to support serialization
    tel = Field(Map(), "telescopes")  


In [3]:
ev = EventContainer()

default values automatically filled in

In [4]:
print(ev.event_id)
print(ev.tels_with_data)
print(ev.tel)

-1
[]
defaultdict(None, {})


In [5]:
print(ev)

{'event_id': -1, 'sub': {'junk': 'nothing'}, 'tel': {}, 'tels_with_data': []}


values can be set as normal for a class:

In [6]:
ev.event_id = 100
ev.event_id

100

In [7]:
ev.as_dict()  # by default only shows the bare items, not sub-containers (See later)

{'event_id': 100, 'sub': __main__.SubContainer:
                           junk: Some junk, 'tel': Map(None,
     {}), 'tels_with_data': []}

In [8]:
ev.as_dict(recursive=True)

{'event_id': 100, 'sub': {'junk': 'nothing'}, 'tel': {}, 'tels_with_data': []}

Now, let's define a sub-container that we can add per telescope:

In [9]:
class TelContainer(Container):
    tel_id = Field(-1, "telescope ID number")
    image = Field(np.zeros(10), "camera pixel data")



and we can add a few of these to the parent container inside the tel dict:

In [10]:
ev.tel[10] = TelContainer()
ev.tel[5] = TelContainer()
ev.tel[42] = TelContainer()

In [11]:
ev.tel

Map(None, {5: __main__.TelContainer:
                             tel_id: telescope ID number
                              image: camera pixel data,
     10: __main__.TelContainer:
                             tel_id: telescope ID number
                              image: camera pixel data,
     42: __main__.TelContainer:
                             tel_id: telescope ID number
                              image: camera pixel data})

### converion to dictionaries

In [12]:
ev.as_dict()

{'event_id': 100, 'sub': __main__.SubContainer:
                           junk: Some junk, 'tel': Map(None,
     {5: __main__.TelContainer:
                              tel_id: telescope ID number
                               image: camera pixel data,
      10: __main__.TelContainer:
                              tel_id: telescope ID number
                               image: camera pixel data,
      42: __main__.TelContainer:
                              tel_id: telescope ID number
                               image: camera pixel data}), 'tels_with_data': []}

In [13]:
ev.as_dict(recursive=True, flatten=False)

{'event_id': 100,
 'sub': {'junk': 'nothing'},
 'tel': {5: {'image': array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]),
   'tel_id': -1},
  10: {'image': array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]),
   'tel_id': -1},
  42: {'image': array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]),
   'tel_id': -1}},
 'tels_with_data': []}

for serialization to a table, we can even flatten the output into a single set of columns

In [14]:
ev.as_dict(recursive=True, flatten=True)

{'event_id': 100,
 'sub_junk': 'nothing',
 'tel_10': {'image': array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]),
  'tel_id': -1},
 'tel_42': {'image': array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]),
  'tel_id': -1},
 'tel_5': {'image': array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]),
  'tel_id': -1},
 'tels_with_data': []}

### setting and clearing values

In [15]:
ev.tel[5].image[:] = 9
print(ev)

{'event_id': 100,
 'sub': {'junk': 'nothing'},
 'tel': {5: {'image': array([ 9.,  9.,  9.,  9.,  9.,  9.,  9.,  9.,  9.,  9.]),
             'tel_id': -1},
         10: {'image': array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]),
              'tel_id': -1},
         42: {'image': array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]),
              'tel_id': -1}},
 'tels_with_data': []}


In [16]:
ev.reset()
ev.as_dict(recursive=True, flatten=True)

{'event_id': -1, 'sub_junk': 'nothing', 'tels_with_data': []}

In [17]:
from ctapipe.io.containers import MCEventContainer

In [18]:
ev = MCEventContainer()

In [19]:
ev

ctapipe.io.containers.MCEventContainer:
                        energy: Monte-Carlo Energy [TeV]
                           alt: Monte-carlo altitude [deg]
                            az: Monte-Carlo azimuth [deg]
                        core_x: MC core position [m]
                        core_y: MC core position [m]
                   h_first_int: Height of first interaction
                        tel[*]: map of tel_id to MCCameraEventContainer